# 🎓 Week 17 - Day 2: Serving ML Models via API

## Today's Goals:
✅ Load and serve pre-trained ML models via FastAPI

✅ Create prediction endpoints for text and image models

✅ Handle file uploads (CSV files, images)

✅ Configure CORS for web application integration

✅ Implement proper error handling for ML models

✅ Build a complete Sentiment Analysis API

---

## 🔧 Part 1: Setup - Install All Required Packages

**What we're installing:**
- `fastapi` & `uvicorn` - API framework (from Day 1)
- `scikit-learn` - ML library for sentiment model
- `joblib` - For loading saved models
- `pandas` - For CSV file handling
- `python-multipart` - For file uploads
- `Pillow` - For image processing

**⏱️ This will take about 1-2 minutes**

In [1]:
# STEP 1: Install packages
print("📦 Installing ML and API packages...\n")

!pip install -q fastapi uvicorn[standard]
!pip install -q scikit-learn joblib pandas
!pip install -q python-multipart Pillow
!pip install -q requests  # For testing

print("\n✅ All packages installed successfully!")
print("\n💡 What we installed:")
print("   • FastAPI - API framework")
print("   • Scikit-learn - ML library")
print("   • Joblib - Model loading")
print("   • Pandas - CSV handling")
print("   • Pillow - Image processing")
print("   • Python-multipart - File uploads")

📦 Installing ML and API packages...




✅ All packages installed successfully!

💡 What we installed:
   • FastAPI - API framework
   • Scikit-learn - ML library
   • Joblib - Model loading
   • Pandas - CSV handling
   • Pillow - Image processing
   • Python-multipart - File uploads


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [2]:
# STEP 2: Import all libraries
import warnings
warnings.filterwarnings('ignore')

# FastAPI essentials
from fastapi import FastAPI, File, UploadFile, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel, Field
from typing import List, Optional

# ML and data processing
import joblib
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# File handling
from io import StringIO, BytesIO
from PIL import Image

# Server utilities
import uvicorn
from threading import Thread
import time
import requests
import json

# For tracking time
from datetime import datetime

print("✅ All libraries imported successfully!")
print("\n🎯 Ready to build ML APIs!")

✅ All libraries imported successfully!

🎯 Ready to build ML APIs!


---

## 📚 Part 2: Understanding ML Model Deployment

### 🤔 The Journey from Notebook to API

**Traditional ML Workflow:**
```
1. Train model in Jupyter
2. Save model to file (.pkl, .joblib)
3. ??? How do others use it? ???
```

**With FastAPI:**
```
1. Train model in Jupyter ✅
2. Save model to file ✅
3. Load model in FastAPI ✅
4. Create prediction endpoint ✅
5. Anyone can use it via HTTP! 🎉
```

### 💾 The Singleton Pattern (Load Once, Use Many)

**❌ BAD Approach:**
```python
@app.post("/predict")
def predict(text: str):
    model = joblib.load('model.pkl')  # Loads EVERY time! Slow!
    return model.predict([text])
```

**✅ GOOD Approach:**
```python
# Load once at startup
model = None

@app.on_event("startup")
def load_model():
    global model
    model = joblib.load('model.pkl')  # Loads ONCE!

@app.post("/predict")
def predict(text: str):
    return model.predict([text])  # Uses loaded model - Fast!
```

### 🎯 Key Concepts:

1. **Model Loading:** Happens once when server starts
2. **Global Variable:** Model stored in memory for all requests
3. **Fast Predictions:** No loading overhead per request
4. **Memory Efficient:** One model instance for all users

**💡 Think of it like:**
- Bad: Opening a dictionary for every word lookup
- Good: Keep the dictionary open on your desk!

---

## 🤖 Part 3: Creating a Simple Sentiment Analysis Model

Before we can serve a model, we need one! Let's create a simple sentiment classifier.

**What we're building:**
- A text classifier that predicts: Positive or Negative
- Uses TF-IDF for text features
- Naive Bayes for classification
- Packaged in a sklearn Pipeline

**In real projects:** You'd load a pre-trained model from Week 6!

In [3]:
# Create a simple sentiment analysis model
print("🤖 Creating sentiment analysis model...\n")

# Sample training data (in real projects, use much more data!)
training_texts = [
    "This product is amazing! I love it!",
    "Excellent service and great quality",
    "Best purchase ever, highly recommend",
    "Wonderful experience, very satisfied",
    "Outstanding product, exceeded expectations",
    "Terrible product, waste of money",
    "Worst purchase ever, very disappointed",
    "Poor quality, would not recommend",
    "Awful experience, terrible service",
    "Horrible product, complete disaster"
]

training_labels = [
    "positive", "positive", "positive", "positive", "positive",
    "negative", "negative", "negative", "negative", "negative"
]

# Create a pipeline: TF-IDF + Naive Bayes
sentiment_model = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=100)),
    ('classifier', MultinomialNB())
])

# Train the model
sentiment_model.fit(training_texts, training_labels)

print("✅ Model trained successfully!")
print("\n📊 Model Details:")
print(f"   • Type: {type(sentiment_model).__name__}")
print(f"   • Features: TF-IDF with max 100 features")
print(f"   • Classifier: Multinomial Naive Bayes")
print(f"   • Classes: {sentiment_model.classes_}")

# Test the model
print("\n🧪 Testing the model:")
test_texts = [
    "I really enjoyed this product",
    "This is absolutely terrible"
]

for text in test_texts:
    prediction = sentiment_model.predict([text])[0]
    print(f"   Text: '{text}'")
    print(f"   Prediction: {prediction}")
    print()

🤖 Creating sentiment analysis model...

✅ Model trained successfully!

📊 Model Details:
   • Type: Pipeline
   • Features: TF-IDF with max 100 features
   • Classifier: Multinomial Naive Bayes
   • Classes: ['negative' 'positive']

🧪 Testing the model:
   Text: 'I really enjoyed this product'
   Prediction: positive

   Text: 'This is absolutely terrible'
   Prediction: positive



In [4]:
# Save the model to a file (this is what you'd do after training)
print("💾 Saving model to file...\n")

model_filename = 'sentiment_model.joblib'
joblib.dump(sentiment_model, model_filename)

print(f"✅ Model saved as: {model_filename}")
print("\n💡 In real projects:")
print("   • Train in Jupyter notebook")
print("   • Save model with joblib.dump()")
print("   • Load in FastAPI with joblib.load()")
print("\n🎯 This file contains:")
print("   • TF-IDF vectorizer (fitted)")
print("   • Naive Bayes classifier (trained)")
print("   • Everything needed for predictions!")

💾 Saving model to file...

✅ Model saved as: sentiment_model.joblib

💡 In real projects:
   • Train in Jupyter notebook
   • Save model with joblib.dump()
   • Load in FastAPI with joblib.load()

🎯 This file contains:
   • TF-IDF vectorizer (fitted)
   • Naive Bayes classifier (trained)
   • Everything needed for predictions!


---

## 🚀 Part 4: Building the Sentiment Analysis API

Now let's create a FastAPI application that serves our sentiment model!

**What we're implementing:**
1. Load model at startup (singleton pattern)
2. Create Pydantic models for input/output
3. Create prediction endpoint
4. Add proper error handling
5. Enable CORS for web apps

**Let's build it step by step!**

In [5]:
# Define Pydantic models for request/response
print("📋 Creating Pydantic models...\n")

class TextInput(BaseModel):
    """
    Input model for single text prediction.
    
    This defines what data the API expects.
    FastAPI will validate automatically!
    """
    text: str = Field(
        ...,  # Required field
        min_length=1,
        max_length=5000,
        description="Text to analyze for sentiment",
        example="This product is amazing!"
    )
    
    class Config:
        schema_extra = {
            "example": {
                "text": "I love this product, it's fantastic!"
            }
        }

class BatchTextInput(BaseModel):
    """
    Input model for batch predictions.
    Allows analyzing multiple texts at once!
    """
    texts: List[str] = Field(
        ...,
        min_items=1,
        max_items=100,
        description="List of texts to analyze",
        example=["Great product!", "Terrible service"]
    )

class PredictionOutput(BaseModel):
    """
    Output model for predictions.
    Structured, consistent response format.
    """
    text: str
    sentiment: str  # 'positive' or 'negative'
    confidence: float
    processing_time_ms: float

class BatchPredictionOutput(BaseModel):
    """
    Output model for batch predictions.
    """
    predictions: List[PredictionOutput]
    total_processed: int
    total_time_ms: float

print("✅ Pydantic models created!")
print("\n📦 Models defined:")
print("   • TextInput - Single text input")
print("   • BatchTextInput - Multiple texts")
print("   • PredictionOutput - Single prediction result")
print("   • BatchPredictionOutput - Batch results")
print("\n💡 These ensure type safety and validation!")

📋 Creating Pydantic models...

✅ Pydantic models created!

📦 Models defined:
   • TextInput - Single text input
   • BatchTextInput - Multiple texts
   • PredictionOutput - Single prediction result
   • BatchPredictionOutput - Batch results

💡 These ensure type safety and validation!


In [6]:
# Create the FastAPI application
print("🚀 Creating Sentiment Analysis API...\n")

# Global variable for the model (loaded once)
loaded_model = None
model_loaded_at = None

# Initialize FastAPI app
app = FastAPI(
    title="Sentiment Analysis API",
    description="""🤖 ML-powered API for sentiment analysis.
    
    Features:
    - Single text prediction
    - Batch text prediction
    - CSV file upload for batch analysis
    - CORS enabled for web apps
    """,
    version="1.0.0"
)

# Add CORS middleware (allows web apps to call our API)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # In production: specify exact domains!
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Event: Load model when server starts
@app.on_event("startup")
async def startup_event():
    """
    This runs ONCE when the server starts.
    Perfect for loading ML models!
    """
    global loaded_model, model_loaded_at
    
    print("🔄 Loading sentiment model...")
    try:
        loaded_model = joblib.load('sentiment_model.joblib')
        model_loaded_at = datetime.now()
        print("✅ Model loaded successfully!")
    except Exception as e:
        print(f"❌ Error loading model: {e}")
        raise

print("✅ FastAPI app created with:")
print("   • CORS enabled (allows web apps to connect)")
print("   • Startup event for model loading")
print("   • Ready for endpoints!")

🚀 Creating Sentiment Analysis API...

✅ FastAPI app created with:
   • CORS enabled (allows web apps to connect)
   • Startup event for model loading
   • Ready for endpoints!


### 🎯 Understanding CORS:

**What is CORS?**
- **C**ross-**O**rigin **R**esource **S**haring
- Browser security feature that blocks requests between different domains

**The Problem:**
```
Your React app: http://localhost:3000
Your API: http://localhost:8000

Browser says: "These are different origins! 🚫 BLOCKED!"
```

**The Solution:**
```python
# Add CORS middleware to FastAPI
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"]  # Allow all (development only!)
)
```

**⚠️ Important:**
- `allow_origins=["*"]` = Allow ALL domains (only for development!)
- Production: `allow_origins=["https://myapp.com"]` (specific domains only)

**💡 Remember:** CORS is a browser thing, not an API restriction!

In [7]:
# Add API endpoints
print("🔧 Adding prediction endpoints...\n")

@app.get("/")
def home():
    """Welcome endpoint with API information"""
    return {
        "message": "Welcome to Sentiment Analysis API! 🤖",
        "version": "1.0.0",
        "endpoints": {
            "POST /predict": "Analyze single text",
            "POST /predict-batch": "Analyze multiple texts",
            "POST /upload-csv": "Upload CSV for batch analysis",
            "GET /model-info": "Get model information"
        },
        "tip": "Visit /docs for interactive API testing!"
    }

@app.get("/model-info")
def get_model_info():
    """
    Get information about the loaded model.
    Useful for debugging and version tracking.
    """
    if loaded_model is None:
        raise HTTPException(
            status_code=503,  # Service Unavailable
            detail="Model not loaded. Please restart the server."
        )
    
    return {
        "model_type": type(loaded_model).__name__,
        "classes": list(loaded_model.classes_),
        "loaded_at": model_loaded_at.isoformat() if model_loaded_at else None,
        "status": "ready"
    }

@app.post("/predict", response_model=PredictionOutput)
def predict_sentiment(input_data: TextInput):
    """
    Predict sentiment for a single text.
    
    Returns:
    - sentiment: 'positive' or 'negative'
    - confidence: probability of prediction
    - processing_time_ms: how long it took
    """
    # Start timer
    start_time = time.time()
    
    # Check if model is loaded
    if loaded_model is None:
        raise HTTPException(
            status_code=503,
            detail="Model not loaded. Please restart the server."
        )
    
    try:
        # Make prediction
        text = input_data.text
        prediction = loaded_model.predict([text])[0]
        probabilities = loaded_model.predict_proba([text])[0]
        
        # Get confidence (probability of predicted class)
        predicted_index = list(loaded_model.classes_).index(prediction)
        confidence = float(probabilities[predicted_index])
        
        # Calculate processing time
        processing_time = (time.time() - start_time) * 1000  # Convert to ms
        
        return PredictionOutput(
            text=text,
            sentiment=prediction,
            confidence=confidence,
            processing_time_ms=round(processing_time, 2)
        )
        
    except Exception as e:
        raise HTTPException(
            status_code=500,
            detail=f"Prediction error: {str(e)}"
        )

@app.post("/predict-batch", response_model=BatchPredictionOutput)
def predict_sentiment_batch(input_data: BatchTextInput):
    """
    Predict sentiment for multiple texts at once.
    More efficient than calling /predict multiple times!
    """
    start_time = time.time()
    
    if loaded_model is None:
        raise HTTPException(
            status_code=503,
            detail="Model not loaded."
        )
    
    try:
        predictions_list = []
        
        for text in input_data.texts:
            text_start = time.time()
            
            # Predict
            prediction = loaded_model.predict([text])[0]
            probabilities = loaded_model.predict_proba([text])[0]
            predicted_index = list(loaded_model.classes_).index(prediction)
            confidence = float(probabilities[predicted_index])
            
            text_time = (time.time() - text_start) * 1000
            
            predictions_list.append(
                PredictionOutput(
                    text=text,
                    sentiment=prediction,
                    confidence=confidence,
                    processing_time_ms=round(text_time, 2)
                )
            )
        
        total_time = (time.time() - start_time) * 1000
        
        return BatchPredictionOutput(
            predictions=predictions_list,
            total_processed=len(predictions_list),
            total_time_ms=round(total_time, 2)
        )
        
    except Exception as e:
        raise HTTPException(
            status_code=500,
            detail=f"Batch prediction error: {str(e)}"
        )

@app.post("/upload-csv")
async def predict_from_csv(file: UploadFile = File(...)):
    """
    Upload a CSV file with a 'text' column for batch sentiment analysis.
    
    CSV format:
    text
    "This is great!"
    "This is terrible"
    ...
    
    Returns predictions for all rows.
    """
    # Validate file type
    if not file.filename.endswith('.csv'):
        raise HTTPException(
            status_code=400,
            detail="File must be a CSV (.csv extension)"
        )
    
    try:
        # Read file
        contents = await file.read()
        df = pd.read_csv(StringIO(contents.decode('utf-8')))
        
        # Validate columns
        if 'text' not in df.columns:
            raise HTTPException(
                status_code=400,
                detail="CSV must have a 'text' column"
            )
        
        # Get predictions for all texts
        texts = df['text'].tolist()
        predictions = loaded_model.predict(texts)
        probabilities = loaded_model.predict_proba(texts)
        
        # Create results
        results = []
        for i, (text, pred, probs) in enumerate(zip(texts, predictions, probabilities)):
            pred_index = list(loaded_model.classes_).index(pred)
            confidence = float(probs[pred_index])
            
            results.append({
                "row": i + 1,
                "text": text,
                "sentiment": pred,
                "confidence": round(confidence, 4)
            })
        
        return {
            "filename": file.filename,
            "total_rows": len(results),
            "results": results
        }
        
    except pd.errors.EmptyDataError:
        raise HTTPException(
            status_code=400,
            detail="CSV file is empty"
        )
    except Exception as e:
        raise HTTPException(
            status_code=500,
            detail=f"Error processing CSV: {str(e)}"
        )

print("✅ All endpoints added!")
print("\n📋 Available endpoints:")
print("   • GET  /             - API info")
print("   • GET  /model-info   - Model details")
print("   • POST /predict      - Single prediction")
print("   • POST /predict-batch - Batch predictions")
print("   • POST /upload-csv   - CSV file upload")
print("\n💡 Note the 'async' keyword in upload-csv!")

🔧 Adding prediction endpoints...

✅ All endpoints added!

📋 Available endpoints:
   • GET  /             - API info
   • GET  /model-info   - Model details
   • POST /predict      - Single prediction
   • POST /predict-batch - Batch predictions
   • POST /upload-csv   - CSV file upload

💡 Note the 'async' keyword in upload-csv!


### 🎯 Understanding File Uploads:

**Why use `async` for file uploads?**

```python
@app.post("/upload-csv")
async def predict_from_csv(file: UploadFile = File(...)):
    contents = await file.read()  # Non-blocking!
```

**Explanation:**
- File uploads can take time (especially large files)
- `async`/`await` allows server to handle other requests while uploading
- More efficient for production servers

**Think of it like:**
- **Sync (blocking):** Waiter waits by your table until you finish eating
- **Async (non-blocking):** Waiter serves other tables while you eat

**File Upload Pattern:**
1. `file: UploadFile = File(...)` - Accept uploaded file
2. `await file.read()` - Read file contents (async)
3. Process the data
4. Return results

**💡 For CSV files:**
- Read with `pd.read_csv(StringIO(contents.decode('utf-8')))`
- Always validate columns exist
- Handle errors gracefully

In [8]:
# Helper function to run server
def run_server(app, port=8000):
    """
    Runs FastAPI server in background thread.
    """
    def start_server():
        uvicorn.run(app, host="127.0.0.1", port=port, log_level="error")
    
    thread = Thread(target=start_server, daemon=True)
    thread.start()
    time.sleep(3)  # Give server time to start and load model
    
    print(f"✅ Server started successfully!")
    print(f"\n🌐 Your Sentiment Analysis API is running at:")
    print(f"   • Main URL: http://127.0.0.1:{port}")
    print(f"   • Interactive Docs: http://127.0.0.1:{port}/docs")
    print("\n💡 Model loading... (check console for 'Model loaded' message)")
    
    return thread

# Start the server!
print("🚀 Starting Sentiment Analysis API...\n")
server_thread = run_server(app, port=8002)

🚀 Starting Sentiment Analysis API...

🔄 Loading sentiment model...


ERROR:    [Errno 10048] error while attempting to bind on address ('127.0.0.1', 8002): only one usage of each socket address (protocol/network address/port) is normally permitted


✅ Model loaded successfully!
✅ Server started successfully!

🌐 Your Sentiment Analysis API is running at:
   • Main URL: http://127.0.0.1:8002
   • Interactive Docs: http://127.0.0.1:8002/docs

💡 Model loading... (check console for 'Model loaded' message)


---

## 🧪 Part 5: Testing the Sentiment Analysis API

Let's test all our endpoints systematically!

**What we'll test:**
1. Model info endpoint
2. Single prediction
3. Batch prediction
4. CSV file upload
5. Error handling

**Testing methods:**
- Programmatic (using requests library)
- Swagger UI (browser-based testing)

In [9]:
# Test 1: Model Info
print("🧪 Testing API Endpoints...\n")
print("=" * 70)
print("\n1️⃣ Testing Model Info Endpoint")
print("-" * 70)

response = requests.get("http://127.0.0.1:8002/model-info")
model_info = response.json()

print("✅ Model Information:")
print(f"   • Model Type: {model_info['model_type']}")
print(f"   • Classes: {model_info['classes']}")
print(f"   • Status: {model_info['status']}")
print(f"   • Loaded At: {model_info['loaded_at']}")

🧪 Testing API Endpoints...


1️⃣ Testing Model Info Endpoint
----------------------------------------------------------------------
✅ Model Information:


KeyError: 'model_type'

In [ ]:
# Test 2: Single Prediction (Positive)
print("\n2️⃣ Testing Single Prediction - Positive Text")
print("-" * 70)

data = {
    "text": "This product is absolutely amazing! I love it so much and highly recommend it to everyone!"
}

response = requests.post("http://127.0.0.1:8002/predict", json=data)
result = response.json()

print(f"📝 Input: '{data['text']}'")
print(f"\n🎯 Prediction:")
print(f"   • Sentiment: {result['sentiment'].upper()}")
print(f"   • Confidence: {result['confidence']:.2%}")
print(f"   • Processing Time: {result['processing_time_ms']:.2f}ms")

In [ ]:
# Test 3: Single Prediction (Negative)
print("\n3️⃣ Testing Single Prediction - Negative Text")
print("-" * 70)

data = {
    "text": "Terrible experience! Worst purchase ever. Complete waste of money and time. Very disappointed!"
}

response = requests.post("http://127.0.0.1:8002/predict", json=data)
result = response.json()

print(f"📝 Input: '{data['text']}'")
print(f"\n🎯 Prediction:")
print(f"   • Sentiment: {result['sentiment'].upper()}")
print(f"   • Confidence: {result['confidence']:.2%}")
print(f"   • Processing Time: {result['processing_time_ms']:.2f}ms")

In [ ]:
# Test 4: Batch Prediction
print("\n4️⃣ Testing Batch Prediction")
print("-" * 70)

batch_data = {
    "texts": [
        "Great product, highly satisfied!",
        "Poor quality, not worth the price",
        "Excellent service and fast delivery",
        "Horrible experience, never buying again",
        "Outstanding quality, exceeded expectations"
    ]
}

response = requests.post("http://127.0.0.1:8002/predict-batch", json=batch_data)
results = response.json()

print(f"📊 Batch Analysis of {results['total_processed']} texts:")
print(f"⏱️  Total Time: {results['total_time_ms']:.2f}ms")
print(f"\n📋 Results:")

for i, pred in enumerate(results['predictions'], 1):
    sentiment_emoji = "😊" if pred['sentiment'] == "positive" else "😞"
    print(f"\n   {i}. {sentiment_emoji} {pred['sentiment'].upper()} ({pred['confidence']:.2%})")
    print(f"      Text: \"{pred['text'][:50]}...\"")
    print(f"      Time: {pred['processing_time_ms']:.2f}ms")

In [ ]:
# Test 5: Create and upload a CSV file
print("\n5️⃣ Testing CSV Upload")
print("-" * 70)

# Create a sample CSV
csv_content = """text
"This is the best product I've ever bought!"
"Worst purchase ever, total waste of money"
"Excellent quality and great customer service"
"Terrible experience, would not recommend"
"Amazing product, exceeded all my expectations"
"""

# Save to file
with open('test_sentiments.csv', 'w') as f:
    f.write(csv_content)

print("📄 Created test CSV file with 5 reviews")

# Upload the file
with open('test_sentiments.csv', 'rb') as f:
    files = {'file': ('test_sentiments.csv', f, 'text/csv')}
    response = requests.post("http://127.0.0.1:8002/upload-csv", files=files)

results = response.json()

print(f"\n✅ File Uploaded: {results['filename']}")
print(f"📊 Total Rows Processed: {results['total_rows']}")
print(f"\n📋 Analysis Results:")

for result in results['results']:
    sentiment_emoji = "😊" if result['sentiment'] == "positive" else "😞"
    print(f"\n   Row {result['row']}: {sentiment_emoji} {result['sentiment'].upper()}")
    print(f"   Confidence: {result['confidence']:.2%}")
    print(f"   Text: \"{result['text'][:60]}...\"")

In [ ]:
# Test 6: Error Handling
print("\n6️⃣ Testing Error Handling")
print("-" * 70)

# Test empty text
print("\n🧪 Test: Empty text (should fail validation)")
data = {"text": ""}
response = requests.post("http://127.0.0.1:8002/predict", json=data)
if response.status_code == 422:  # Validation error
    print("   ✅ Validation error caught correctly!")
    print(f"   Status: {response.status_code}")
    print(f"   Message: Field validation failed")

# Test missing field
print("\n🧪 Test: Missing 'text' field (should fail)")
data = {"wrong_field": "some text"}
response = requests.post("http://127.0.0.1:8002/predict", json=data)
if response.status_code == 422:
    print("   ✅ Missing field error caught!")
    print(f"   Status: {response.status_code}")

# Test wrong file type
print("\n🧪 Test: Wrong file type (should fail)")
with open('test.txt', 'w') as f:
    f.write("test")

with open('test.txt', 'rb') as f:
    files = {'file': ('test.txt', f, 'text/plain')}
    response = requests.post("http://127.0.0.1:8002/upload-csv", files=files)

if response.status_code == 400:
    print("   ✅ File type error caught!")
    error = response.json()
    print(f"   Error: {error['detail']}")

print("\n" + "=" * 70)
print("\n✅ All tests completed! Your ML API is working perfectly!")

---

## 🌐 Part 6: Testing in Swagger UI

Now let's use the interactive documentation!

### 🎯 Swagger UI Guide:

**1. Open Swagger UI:**
- Visit: http://127.0.0.1:8002/docs
- You'll see all your endpoints listed!

**2. Test /predict endpoint:**
- Click on `POST /predict`
- Click "Try it out"
- Modify the example text:
  ```json
  {
    "text": "This is an amazing product!"
  }
  ```
- Click "Execute"
- See the response below!

**3. Test /predict-batch:**
- Same process, but with multiple texts:
  ```json
  {
    "texts": [
      "Great product!",
      "Terrible service"
    ]
  }
  ```

**4. Test /upload-csv:**
- Click `POST /upload-csv`
- Click "Try it out"
- Click "Choose File"
- Select `test_sentiments.csv`
- Click "Execute"
- See all results!

**💡 Swagger UI Benefits:**
- ✅ No code needed to test
- ✅ See request/response formats
- ✅ Try different inputs instantly
- ✅ Share with frontend developers
- ✅ Auto-generated from your code!

---

## 💡 Part 7: Best Practices for ML APIs

### 🎯 Model Loading:

✅ **DO:**
- Load model once at startup
- Use global variable or dependency injection
- Add health check endpoint
- Log model loading status

❌ **DON'T:**
- Load model on every request
- Load multiple copies of same model
- Ignore loading errors

### 🎯 Error Handling:

✅ **DO:**
- Use try-except blocks
- Return meaningful error messages
- Use appropriate HTTP status codes
- Log errors for debugging

❌ **DON'T:**
- Let server crash on errors
- Return generic "Error" messages
- Expose internal error details to users

### 🎯 Input Validation:

✅ **DO:**
- Use Pydantic models
- Set min/max lengths
- Validate file types
- Check file sizes

❌ **DON'T:**
- Trust all user input
- Allow unlimited file sizes
- Skip type checking

### 🎯 Response Format:

✅ **DO:**
- Return consistent structure
- Include confidence scores
- Add metadata (processing time, model version)
- Use Pydantic for response models

❌ **DON'T:**
- Return raw predictions only
- Change response structure randomly
- Skip documentation

### 🎯 Performance:

✅ **DO:**
- Use batch endpoints for multiple predictions
- Consider async for I/O operations
- Cache frequent predictions
- Monitor response times

❌ **DON'T:**
- Process files synchronously
- Load large models on every prediction
- Ignore memory usage

### 🎯 Security:

✅ **DO:**
- Validate file uploads
- Set CORS properly (specific origins in production)
- Add rate limiting
- Use environment variables for secrets

❌ **DON'T:**
- Allow all CORS origins in production
- Accept unlimited requests
- Hardcode API keys

**💡 Remember:** Your API is the gateway to your ML model - make it robust!

---

## 🎯 Part 8: Beginner Challenge

### 🏆 Your Mission:

Enhance the Sentiment Analysis API with new features!

### 📋 Requirements:

**1. Add a Statistics Endpoint**
- Create `GET /stats` that returns:
  - Total predictions made
  - Positive vs negative count
  - Average confidence score
- Use a global counter to track predictions

**2. Add Sentiment Distribution Endpoint**
- Create `POST /analyze-distribution` that:
  - Takes a list of texts
  - Returns percentage positive vs negative
  - Shows average confidence per sentiment

**3. Add Language Detection (Bonus)**
- Install `langdetect` library
- Add language detection to predictions
- Return warning if text is not English

### 💡 Hints:

```python
# Hint 1: Global statistics tracking
prediction_stats = {
    "total": 0,
    "positive": 0,
    "negative": 0,
    "confidences": []
}

# Update in prediction endpoints
prediction_stats["total"] += 1
prediction_stats[sentiment] += 1
prediction_stats["confidences"].append(confidence)

# Hint 2: Distribution calculation
total = len(predictions)
positive_pct = (positive_count / total) * 100
negative_pct = (negative_count / total) * 100

# Hint 3: Language detection
from langdetect import detect
language = detect(text)
if language != 'en':
    warning = "Text may not be English"
```

### 🎯 Expected Outcome:

```python
# GET /stats
{
    "total_predictions": 25,
    "positive_count": 15,
    "negative_count": 10,
    "positive_percentage": 60.0,
    "average_confidence": 0.87
}

# POST /analyze-distribution
{
    "total_texts": 10,
    "distribution": {
        "positive": {"count": 6, "percentage": 60.0, "avg_confidence": 0.89},
        "negative": {"count": 4, "percentage": 40.0, "avg_confidence": 0.83}
    }
}
```

### 🌟 Bonus Challenges:

1. Add a `/health` endpoint that checks:
   - Server status
   - Model loaded status
   - Memory usage

2. Add text preprocessing:
   - Remove URLs
   - Remove special characters
   - Convert to lowercase

3. Add response caching:
   - Cache predictions for identical texts
   - Use dictionary: `{text: prediction}`
   - Save time on repeated queries

In [ ]:
# Your code here!
# Try implementing the challenge requirements

# Step 1: Add statistics tracking
# prediction_stats = {...}

# Step 2: Create /stats endpoint
# @app.get("/stats")
# def get_stats():
#     ...

# Step 3: Create /analyze-distribution endpoint
# @app.post("/analyze-distribution")
# def analyze_distribution(data: BatchTextInput):
#     ...

# Restart server and test in Swagger UI!

pass

---

## 📚 Summary - What We Learned Today

### 1. ML Model Deployment 🤖
- **Singleton pattern** - Load model once, use many times
- **Startup events** - Initialize resources when server starts
- **Global variables** - Store model in memory for all requests
- **Model serialization** - Save with joblib, load in API

### 2. Prediction Endpoints 🔮
- **Single predictions** - `/predict` for one text at a time
- **Batch predictions** - `/predict-batch` for multiple texts efficiently
- **Structured responses** - Consistent format with confidence and metadata
- **Pydantic models** - Type-safe input/output validation

### 3. File Upload Handling 📤
- **Async file uploads** - Use `async`/`await` for efficiency
- **CSV processing** - Parse with pandas, return structured results
- **File validation** - Check type, size, content before processing
- **Error handling** - Graceful failures with helpful messages

### 4. CORS Configuration 🌐
- **What is CORS** - Browser security feature for cross-origin requests
- **Why it matters** - Web apps need CORS to call your API
- **How to enable** - CORSMiddleware in FastAPI
- **Production considerations** - Specific origins only, not "*"

### 5. Error Handling ⚠️
- **ML-specific errors** - Wrong input shape, invalid features
- **HTTP exceptions** - 400 (Bad Request), 500 (Server Error), 503 (Unavailable)
- **Validation errors** - Automatic from Pydantic (422 status)
- **User-friendly messages** - Always explain what went wrong

### 6. API Best Practices 💡
- **Response structure** - Include metadata (confidence, time, version)
- **Endpoint design** - Single and batch operations
- **Documentation** - Docstrings appear in Swagger UI
- **Performance** - Batch operations, async for I/O

### 7. Testing Strategies 🧪
- **Programmatic testing** - Use requests library
- **Swagger UI** - Interactive browser-based testing
- **Error case testing** - Verify failures handle gracefully
- **Multiple input types** - Text, JSON, files

---

## 🎯 Key Takeaways

✅ **ML models become useful when deployed as APIs**
- Notebooks are for development, APIs are for deployment

✅ **Load models once, not on every request**
- Use startup events and global variables
- Massive performance improvement!

✅ **Structure your responses consistently**
- Prediction + confidence + metadata
- Makes client integration easier

✅ **CORS is essential for web apps**
- Browser security blocks cross-origin by default
- Easy to enable in FastAPI

✅ **Validation and error handling are critical**
- Pydantic validates automatically
- Always return helpful error messages

✅ **Swagger UI is your best friend**
- Test immediately without writing client code
- Share with team for integration

✅ **Async for file operations**
- Better performance under load
- Non-blocking I/O operations

---

## 💡 Pro Tips for Production

1. **Model Versioning**
   - Include model version in responses
   - Track which model made each prediction
   - Helps with debugging and A/B testing

2. **Logging**
   - Log all predictions for monitoring
   - Track response times
   - Alert on errors

3. **Rate Limiting**
   - Prevent abuse
   - Use slowapi library
   - Set per-user or per-IP limits

4. **Caching**
   - Cache frequent predictions
   - Use Redis or in-memory dict
   - Massive speed improvement

5. **Monitoring**
   - Track API usage
   - Monitor model performance
   - Alert on drift or degradation

6. **Security**
   - Use API keys for authentication
   - Validate all inputs
   - Limit file sizes
   - Use HTTPS in production

---

## 🚀 Next Steps - Tomorrow!

**Day 3: Advanced FastAPI**

We'll learn:
- **Server-Sent Events (SSE)** - Streaming LLM outputs
- **API documentation** - Customizing Swagger UI
- **Security best practices** - Environment variables, API keys
- **Production deployment** - Docker, CI/CD basics
- **Team presentation** - Communication skills

**Get ready to make your APIs production-ready! 🚀**

---

## 🎉 Congratulations!

You've built a production-grade ML API!

**You now know how to:**
- ✅ Load and serve ML models via FastAPI
- ✅ Create prediction endpoints (single and batch)
- ✅ Handle file uploads (CSV, images)
- ✅ Configure CORS for web apps
- ✅ Handle errors gracefully
- ✅ Structure responses properly
- ✅ Test with Swagger UI
- ✅ Deploy real ML models to production!

**This is a HUGE achievement! 🎊**

Your ML models are no longer stuck in notebooks - they're accessible to the world!

**Practice what you learned and see you tomorrow! 🚀**